follow from Fork Repository

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]    = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
import numpy                     as     np
import matplotlib.pyplot         as     plt
import math
from keras.layers                import Input, GlobalAveragePooling2D, Concatenate
from keras.layers.core           import Dense, Activation, Flatten
from keras.layers.normalization  import BatchNormalization
from keras.layers.convolutional  import Conv2D, MaxPooling2D, AveragePooling2D
from keras.models                import Model
from keras.datasets              import fashion_mnist
from keras.preprocessing.image   import ImageDataGenerator
from tensorflow.python.client    import device_lib
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters       import gaussian_filter


In [ ]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())


In [ ]:
class DenseNet121(object):
    '''
    "Densely Connected Convolutional Networks"
    Gao Huang et al.
    https://arxiv.org/abs/1608.06993
    '''
    def __init__(self, input_shape, output_dim, k=32, theta=0.5):
        '''
        # Arguments
            k:     growth rate
            theta: compression rate
        '''
        self.k = k
        self.theta = theta

        x = Input(shape=input_shape)
        h = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(x)
        h = BatchNormalization()(h)
        h = Activation('relu')(h)
        h = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(h)
        h, n_channel = self._dense_block(h, 64, 6)
        h, n_channel = self._transition(h, n_channel)
        h, n_channel = self._dense_block(h, n_channel, 12)
        h, n_channel = self._transition(h, n_channel)
        h, n_channel = self._dense_block(h, n_channel, 24)
        h, n_channel = self._transition(h, n_channel)
        h, _ = self._dense_block(h, n_channel, 16)
        h = GlobalAveragePooling2D()(h)
        h = Dense(1000, activation='relu')(h)
        y = Dense(output_dim, activation='softmax')(h)
        self.model = Model(x, y)

    def __call__(self):
        return self.model

    def _dense_block(self, x, n_channel, nb_blocks):
        h = x
        for _ in range(nb_blocks):
            stream = h
            h = BatchNormalization()(h)
            h = Activation('relu')(h)
            h = Conv2D(128, kernel_size=(1, 1), padding='same')(h)
            h = BatchNormalization()(h)
            h = Activation('relu')(h)
            h = Conv2D(self.k, kernel_size=(3, 3), padding='same')(h)
            h = Concatenate()([stream, h])
            n_channel += self.k

        return h, n_channel

    def _transition(self, x, n_channel):
        n_channel = int(n_channel * self.theta)
        h = BatchNormalization()(x)
        h = Activation('relu')(h)
        h = Conv2D(n_channel, kernel_size=(1, 1), padding='same')(h)
        return AveragePooling2D()(h), n_channel


if __name__ == '__main__':
    '''
    Build model
    '''
    densenet = DenseNet121((32, 32, 3), 10)
    model = densenet()
    model.summary()


In [ ]:
from keras.datasets import cifar10
from keras.utils import to_categorical

# CIFAR-10の読み込み
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('np.shape(X_train) = (%d, %d, %d, %d)' % np.shape(X_train))
print('np.shape(y_train) = (%d, %d)' % np.shape(y_train))
print('np.shape(X_test)  = (%d, %d, %d, %d)' % np.shape(X_test))
print('np.shape(y_test)  = (%d, %d)' % np.shape(y_test))


In [ ]:
# categorical y ...
y_train, y_test = to_categorical(y_train), to_categorical(y_test)


In [ ]:
# back up by deep copy
X_train_org = X_train.copy()
y_train_org = y_train.copy()


In [ ]:
# make class for data augmentation
datagen = ImageDataGenerator(rotation_range      = 20,
                             width_shift_range   = 0.2,
                             height_shift_range  = 0.2,
                             zoom_range          = 0.2,
                             channel_shift_range = 50,
                             horizontal_flip     = True, 
                             fill_mode           = 'reflect')


In [ ]:
# exec augmentation
aug_num     = 10 # num of augmentation images from one original image 
watch_num   = 50 # plot aug image num

watch_idx   = np.random.permutation(len(X_train_org))[:watch_num]
X_train_aug = np.zeros((np.shape(X_train_org) * np.array([aug_num, 1, 1, 1])), dtype='uint8')
y_train_aug = np.zeros((np.shape(y_train_org) * np.array([aug_num, 1])),       dtype='float32')
aug_i       = 0

for img_i in range(len(X_train_org)):

    img_tmp      = X_train_org[img_i:(img_i + 1), :, :, :] # (Height, Width, Channels)  -> (1, Height, Width, Channels) 
    datagen_flow = datagen.flow(img_tmp, batch_size=1) # 1枚しかないので、ミニバッチ数は1
    
    watch_flg    = (np.sum(img_i == watch_idx) == 1)
    
    if (watch_flg):
        # Python ジェネレーターで9枚生成して、表示する。
        plt.figure(figsize=(10, int(2 * math.ceil(aug_num / 6))))
        plt.subplot(math.ceil(aug_num / 6), 6, 1)
        plt.imshow(img_tmp[0, :, :, :].astype('uint8'))
        plt.axis('off')
        plt.title('y = %d, img_i = %d' % (np.dot(y_train_org[img_i], (np.arange(len(y_train_org[0])) + 1)), img_i))
    
    for aug_j in range(aug_num):
        batches  = next(datagen_flow)  # (NumBatches, Height, Width, Channels) の4次元データを返す。
        gen_img  = batches[0].astype(np.uint8)
        X_train_aug[aug_i, :, :, :] = gen_img
        y_train_aug[aug_i, :]       = y_train_org[img_i:(img_i + 1), :]

        if (watch_flg):
            plt.subplot(math.ceil(aug_num / 6), 6, aug_j + 2)
            plt.imshow(X_train_aug[aug_i, :, :, :])
            plt.axis('off')
            plt.title('aug_j = %d' % aug_j)

        aug_i    = aug_i + 1
         
    if (watch_flg):
        print('----- (img_i == %d, progress:%.2f%%) -----' % (img_i, (img_i / len(X_train_org) * 100)))
        plt.show()

print('augmentarion finished')


In [ ]:
# add augmentation data
X_train = np.concatenate([X_train, X_train_aug], axis=0)
y_train = np.concatenate([y_train, y_train_aug], axis=0)

print('np.shape(X_train) = (%d, %d, %d, %d)' % np.shape(X_train))
print('np.shape(y_train) = (%d, %d)'         % np.shape(y_train))


In [ ]:
# https://gist.github.com/ShangxuanWu/f53f9d2ca9338f4f13b466509cb07947

# Function to distort image
def elastic_transform(image, alpha, sigma, random_state=None):
    
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    random_state = np.random.RandomState(None)

    shape   = image.shape
    dx      = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy      = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dz      = np.zeros_like(dx)

    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    distored_image = map_coordinates(image, indices, order=3, mode='reflect')
    return distored_image.reshape(image.shape)


In [ ]:
# exec augmentation (elastic distortion)
aug_num     = 5  # num of augmentation images from one original image 
watch_num   = 30 # plot aug image num

watch_idx   = np.random.permutation(len(X_train_org))[:watch_num]
X_train_aug = np.zeros((np.shape(X_train_org) * np.array([aug_num, 1, 1, 1])), dtype='uint8')
y_train_aug = np.zeros((np.shape(y_train_org) * np.array([aug_num, 1])),       dtype='float32')
aug_i       = 0

alpha_ind   = 1.5
sigma_ind   = 0.09

for img_i in range(len(X_train_org)):

    img_tmp      = X_train_org[img_i:(img_i + 1), :, :, :] # (1, Height, Width, Channels) 
    datagen_flow = datagen.flow(img_tmp, batch_size=1) # batch_size is np.shape(img_tmp)[0]
    
    watch_flg    = (np.sum(img_i == watch_idx) == 1)
    
    if (watch_flg):
        plt.figure(figsize=(10, int(5 * math.ceil(aug_num / 6))))
        plt.subplot(math.ceil(aug_num / 3), 3, 1)
        plt.imshow(img_tmp[0, :, :, :].astype('uint8'))
        plt.axis('off')
        plt.title('y = %d, img_i = %d' % (np.dot(y_train_org[img_i], (np.arange(len(y_train_org[0])) + 1)), img_i))
    
    for aug_j in range(aug_num):
        batches  = next(datagen_flow)  # (NumBatches, Height, Width, Channels)
        gen_img  = batches[0].astype(np.uint8)
        gen_img  = elastic_transform(image=gen_img, alpha=(gen_img.shape[1] * alpha_ind), sigma=(gen_img.shape[1] * sigma_ind))
        X_train_aug[aug_i, :, :, :] = gen_img
        y_train_aug[aug_i, :]       = y_train_org[img_i:(img_i + 1), :]

        if (watch_flg):
            plt.subplot(math.ceil(aug_num / 3), 3, aug_j + 2)
            plt.imshow(X_train_aug[aug_i, :, :, :])
            plt.axis('off')
            plt.title('aug_j = %d' % aug_j)

        aug_i    = aug_i + 1
         
    if (watch_flg):
        print('----- (img_i == %d, progress:%.2f%%) -----' % (img_i, (img_i / len(X_train_org) * 100)))
        plt.show()

print('augmentarion finished')


In [ ]:
# add augmentation data
X_train = np.concatenate([X_train, X_train_aug], axis=0)
y_train = np.concatenate([y_train, y_train_aug], axis=0)

print('np.shape(X_train) = (%d, %d, %d, %d)' % np.shape(X_train))
print('np.shape(y_train) = (%d, %d)'         % np.shape(y_train))


In [ ]:
# compile ...
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])


In [ ]:
# calc scaling parameter ... (※augmentation前のデータで計算した方が良いかも)
X_train_mean = np.mean(X_train)
X_train_std  = np.std(X_train)


In [ ]:
# scaling (normalize) ...
X_train = (X_train - X_train_mean) / (X_train_std + 1e-20)
X_test  = (X_test  - X_train_mean) / (X_train_std + 1e-20)


In [ ]:
# learning ... (※メモリ問題が深刻になってきたら、augmentationやりながらのpartial fitに要切替)
model.fit(x               = X_train, 
          y               = y_train, 
          batch_size      = 16, 
          epochs          = 50, 
          verbose         = 1, 
          validation_data = (X_test, y_test))
